Next steps: Custom softmax & other improvements, MultiplexNet

Check out Decision Transformer: Reinforcement Learning via Sequence Modeling

In [12]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
try:
    print(curr_dir)
except:
    curr_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    os.chdir(curr_dir)
    print(curr_dir)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL


In [6]:
import numpy as np
import pandas as pd

import torch as th
from torch.optim import Adam
from torcheval.metrics import MeanSquaredError
from training_loop import TrainingLoop, SimpleTrainingStep
from training_loop.callbacks import EarlyStopping, TensorBoardLogger
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from envs import BlendEnv, reconstruct_dict

import datetime
from models import IL_MLP_simple, IL_RNN_simple

In [2]:
import pickle
dataset_path = f'./decision-transformer/gym/data/blend-medium-v4.pkl'
with open(dataset_path, 'rb') as f:
    trajectories = pickle.load(f)

In [3]:
print([(i,k) for i,k in enumerate(['sources_s1', 'sources_s2', 'blenders_j1', 'blenders_j2', 'blenders_j3', 'blenders_j4', 'demands_p1', 'demands_p2', 
       'sources_avail_next_0_s1', 'sources_avail_next_0_s2', 'demands_avail_next_0_p1', 'demands_avail_next_0_p2',])])
print([(i,k) for i,k in enumerate([
    'source_blend_s1_j1', 'source_blend_s1_j2', 'source_blend_s1_j3', 'source_blend_s1_j4',
    'source_blend_s2_j1', 'source_blend_s2_j2', 'source_blend_s2_j3', 'source_blend_s2_j4',
    'blend_demand_j1_p1', 'blend_demand_j1_p2',
    'blend_demand_j2_p1', 'blend_demand_j2_p2',
    'blend_demand_j3_p1', 'blend_demand_j3_p2',
    'blend_demand_j4_p1', 'blend_demand_j4_p2',
    'tau_s1', 'tau_s2',
    'delta_p1', 'delta_p2'
])], "\n")
traj = trajectories[0]
for t in range(traj["observations"].shape[0]):
    print("t =",t)
    print([(i,k) for i,k in enumerate(traj["observations"][t][:8])], [(i+8,k) for i,k in enumerate(traj["observations"][t][12:16])])
    print([(i,k) for i,k in enumerate(traj["actions"][t])])
    print(traj["rewards"][t],"\n")

[(0, 'sources_s1'), (1, 'sources_s2'), (2, 'blenders_j1'), (3, 'blenders_j2'), (4, 'blenders_j3'), (5, 'blenders_j4'), (6, 'demands_p1'), (7, 'demands_p2'), (8, 'sources_avail_next_0_s1'), (9, 'sources_avail_next_0_s2'), (10, 'demands_avail_next_0_p1'), (11, 'demands_avail_next_0_p2')]
[(0, 'source_blend_s1_j1'), (1, 'source_blend_s1_j2'), (2, 'source_blend_s1_j3'), (3, 'source_blend_s1_j4'), (4, 'source_blend_s2_j1'), (5, 'source_blend_s2_j2'), (6, 'source_blend_s2_j3'), (7, 'source_blend_s2_j4'), (8, 'blend_demand_j1_p1'), (9, 'blend_demand_j1_p2'), (10, 'blend_demand_j2_p1'), (11, 'blend_demand_j2_p2'), (12, 'blend_demand_j3_p1'), (13, 'blend_demand_j3_p2'), (14, 'blend_demand_j4_p1'), (15, 'blend_demand_j4_p2'), (16, 'tau_s1'), (17, 'tau_s2'), (18, 'delta_p1'), (19, 'delta_p2')] 

t = 0
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0)] [(8, 0.0), (9, 25.9), (10, 0.0), (11, 0.0)]
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7

In [55]:
idx = [12, 16, 20, 24, 28, 32]
s1 = list(float(traj["observations"][k][[12]]) for k in range(6))
s2 = list(float(traj["observations"][k][[13]]) for k in range(6))
p1 = list(float(traj["observations"][k][[14]]) for k in range(6))
p2 = list(float(traj["observations"][k][[15]]) for k in range(6))

tau0   = {"s1": s1, "s2": s2}
delta0 = {"p1": p1, "p2": p2}

In [56]:
tau0, delta0

({'s1': [0.0, 12.9, 14.5, 13.5, 0.0, 0.0],
  's2': [25.9, 25.1, 11.6, 0.0, 11.9, 0.0]},
 {'p1': [0.0, 0.0, 12.3, 21.7, 0.0, 13.4],
  'p2': [0.0, 19.5, 18.7, 16.0, 23.5, 20.6]})

In [58]:
M=P=Q=B=D=0
Z=1
import json
with open("./configs/json/action_sample_simple.json" ,"r") as f:
    action = f.readline()
action_sample = json.loads(action)
with open("./configs/json/connections_simple.json" ,"r") as f:
    connections_s = f.readline()
connections = json.loads(connections_s)
env = BlendEnv(v=True, connections = connections, action_sample=action_sample, 
               M=M, P=P, Q=Q, B=B, Z=Z, D=D, tau0 = tau0, delta0 = delta0)

In [60]:
obs = env.reset()
obs, obs_dict = obs
prev_rew = 0
for t in range(env.T):
    print("\nt:",t)
    print("t_obs:", obs[-1])
    dobs = reconstruct_dict(obs, env.mapping_obs)
    datasetobs = reconstruct_dict(traj["observations"][t], env.mapping_obs)
    
    print("\n  DATASET OBS  s,b,d:",datasetobs["sources"], datasetobs["blenders"], datasetobs["demands"], "\n               tau, delta:",datasetobs["sources_avail_next_0"], datasetobs["demands_avail_next_0"])
    print("\n  OBS          s,b,d:",dobs["sources"], dobs["blenders"], dobs["demands"], "\n               tau, delta:",dobs["sources_avail_next_0"], dobs["demands_avail_next_0"])
    
    action = traj["actions"][t]
    dact = reconstruct_dict(action, env.mapping_act)
    
    print("\n   ACTION : ")
    print("tau, delta:", dact["tau"], dact["delta"])
    print("source_blend", dact["source_blend"])
    print("blend_demand", dact["blend_demand"])
    
    obs, reward, done, term, dict_obs = env.step(action)
    print("\n NEW OBS   >>     ",dict_obs["dict_state"]["sources"], dict_obs["dict_state"]["blenders"], dict_obs["dict_state"]["demands"])
    print("   " ,reward - prev_rew," / ", traj["rewards"][t])
    prev_rew = reward


t: 0
t_obs: 0.0

  DATASET OBS  s,b,d: {'s1': 0.0, 's2': 0.0} {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 0.0} {'p1': 0.0, 'p2': 0.0} 
               tau, delta: {'s1': 0.0, 's2': 25.9} {'p1': 0.0, 'p2': 0.0}

  OBS          s,b,d: {'s1': 0.0, 's2': 0.0} {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 0.0} {'p1': 0.0, 'p2': 0.0} 
               tau, delta: {'s1': 0.0, 's2': 25.9} {'p1': 0.0, 'p2': 0.0}

   ACTION : 
tau, delta: {'s1': 0.0, 's2': 25.9} {'p1': 0.0, 'p2': 0.0}
source_blend {'s1': {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 0.0}, 's2': {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 25.9}}
blend_demand {'j1': {'p1': 0.0, 'p2': 0.0}, 'j2': {'p1': 0.0, 'p2': 0.0}, 'j3': {'p1': 0.0, 'p2': 0.0}, 'j4': {'p1': 0.0, 'p2': 0.0}}
j1: inv: 0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j2: inv: 0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
j3: inv: 0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0


In [3]:
now = datetime.datetime.now().strftime('%m%d-%H%M')
modelname = f"MLP_{now}"

In [11]:
cols_obs = [
    'sources_s1', 'sources_s2',
    'blenders_j1', 'blenders_j2', 'blenders_j3', 'blenders_j4',
    'demands_p1', 'demands_p2',
    'properties_j1_q1', 'properties_j2_q1', 'properties_j3_q1', 'properties_j4_q1',
    'sources_avail_next_0_s1', 'sources_avail_next_0_s2', 'demands_avail_next_0_p1', 'demands_avail_next_0_p2',
    'sources_avail_next_1_s1', 'sources_avail_next_1_s2', 'demands_avail_next_1_p1', 'demands_avail_next_1_p2',
    'sources_avail_next_2_s1', 'sources_avail_next_2_s2', 'demands_avail_next_2_p1', 'demands_avail_next_2_p2',
    'sources_avail_next_3_s1', 'sources_avail_next_3_s2', 'demands_avail_next_3_p1', 'demands_avail_next_3_p2',
    'sources_avail_next_4_s1', 'sources_avail_next_4_s2', 'demands_avail_next_4_p1', 'demands_avail_next_4_p2',
    'sources_avail_next_5_s1', 'sources_avail_next_5_s2', 'demands_avail_next_5_p1', 'demands_avail_next_5_p2',
    't'
]
cols_act = [
    'source_blend_s1_j1', 'source_blend_s1_j2', 'source_blend_s1_j3', 'source_blend_s1_j4',
    'source_blend_s2_j1', 'source_blend_s2_j2', 'source_blend_s2_j3', 'source_blend_s2_j4',
    'blend_demand_j1_p1', 'blend_demand_j1_p2',
    'blend_demand_j2_p1', 'blend_demand_j2_p2',
    'blend_demand_j3_p1', 'blend_demand_j3_p2',
    'blend_demand_j4_p1', 'blend_demand_j4_p2',
    'tau_s1', 'tau_s2',
    'delta_p1', 'delta_p2'
]

In [12]:
X, y = pd.DataFrame([], columns=cols_obs), pd.DataFrame([], columns=cols_act)

for file in os.listdir("./data/simple"):
    df = pd.read_csv(os.path.join("./data/simple", file))
    if "OBS" in file:
        X = pd.concat([X, df])
    elif "ACT" in file:
        y = pd.concat([y, df])

X = X.drop("Unnamed: 0", axis=1)
y = y.drop("Unnamed: 0", axis=1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# Normalization to [0, 1]
X_min = X.min()
y_min = y.min()
X_max = X.max()
y_max = y.max()

def norm_X(X):
    return (X - X_min) / (X_max - X_min)
def denorm_X(X):
    return X * (X_max - X_min) + X_min
def norm_y(y):
    return (y - y_min) / (y_max - y_min)
def denorm_y(y):
    return y * (y_max - y_min) + y_min

X_train = norm_X(X_train)
X_val = norm_X(X_val)
X_test = norm_X(X_test)
y_train = norm_y(y_train)
y_val = norm_y(y_val)
y_test = norm_y(y_test)

In [18]:
class MyData(Dataset):
    def __init__(self, df_X: pd.DataFrame, df_y: pd.DataFrame) -> None:
        super().__init__()
        self.df_X = df_X
        self.df_y = df_y
        
    def __getitem__(self, index):
        return(th.Tensor(self.df_X.iloc[index]), th.Tensor(self.df_y.iloc[index]))
    
    def __len__(self):
        return(self.df_X.shape[0])

In [19]:
train_ds = MyData(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=128)
val_ds = MyData(X_val, y_val)
val_dl = DataLoader(val_ds, batch_size=128)
test_ds = MyData(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=128)
len(train_ds)

48027

In [20]:
model = IL_RNN_simple()

In [21]:
loop = TrainingLoop(
    model,
    step=SimpleTrainingStep(
        optimizer_fn = lambda params: Adam(params, lr=1e-4),
        loss = th.nn.MSELoss(),
        # metrics = ('MSE', MeanSquaredError()),
    ),
    device='cpu'
)
train_loss, val_loss = loop.fit(
    train_dl,
    val_dl,
    epochs=100,
    callbacks=[EarlyStopping(monitor='val_loss', mode='min', patience=20), TensorBoardLogger(f"./logs/simple/imit/{modelname}")],
)

Epoch 1/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 2/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 3/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 4/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 5/100:   0%|          | 0/423 [00:00<?, ?it/s]

...

Epoch 97/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 98/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 99/100:   0%|          | 0/423 [00:00<?, ?it/s]

Epoch 100/100:   0%|          | 0/423 [00:00<?, ?it/s]

(                 loss  batch  epoch
 epoch batch                        
 1      1     0.029406      1      1
        2     0.029751      2      1
        3     0.028815      3      1
        4     0.031082      4      1
        5     0.031483      5      1
 ...               ...    ...    ...
 100    373   0.009779    373    100
        374   0.009782    374    100
        375   0.009776    375    100
        376   0.009770    376    100
       -1     0.009770     -1    100
 
 [37700 rows x 3 columns],
                      val_loss  val_batch  val_epoch
 val_epoch val_batch                                
 1          1         0.033698          1          1
            2         0.031478          2          1
            3         0.031774          3          1
            4         0.032318          4          1
            5         0.032689          5          1
 ...                       ...        ...        ...
 100        44        0.010145         44        100
            4

In [32]:
modelname

'MLP_0916-1442'

In [34]:
th.save(model.state_dict(), f"weights/imitation/{modelname}.pth")

In [6]:
model = IL_RNN_simple()
model.load_state_dict( th.load("weights/imitation/MLP_0916-1442.pth") )

<All keys matched successfully>

In [39]:
obs = env.reset()
obs, obs_dict = obs
obs_dict

{'dict_state': {'sources': {'s1': 0, 's2': 0},
  'blenders': {'j1': 0, 'j2': 0, 'j3': 0, 'j4': 0},
  'demands': {'p1': 0, 'p2': 0},
  'properties': {'j1': {'q1': 0},
   'j2': {'q1': 0},
   'j3': {'q1': 0},
   'j4': {'q1': 0}},
  'sources_avail_next_0': {'s1': 0.0, 's2': 19.986952261513},
  'demands_avail_next_0': {'p1': 13.147715601294175, 'p2': 0.0},
  'sources_avail_next_1': {'s1': 13.966896800299835, 's2': 16.027758933172862},
  'demands_avail_next_1': {'p1': 13.442337106310015, 'p2': 0.0},
  'sources_avail_next_2': {'s1': 0.0, 's2': 0.0},
  'demands_avail_next_2': {'p1': 0.0, 'p2': 12.955960410278145},
  'sources_avail_next_3': {'s1': 13.208133523830055, 's2': 16.733810795624873},
  'demands_avail_next_3': {'p1': 0.0, 'p2': 0.0},
  'sources_avail_next_4': {'s1': 14.629419833295623, 's2': 13.002356622394464},
  'demands_avail_next_4': {'p1': 11.062252337464116, 'p2': 16.85468203066291},
  'sources_avail_next_5': {'s1': 15.339391058369007, 's2': 11.68016330073389},
  'demands_avail_n

In [170]:
tau0   = {s: [np.random.binomial(1, 0.7) * np.random.normal(15, 2) for _ in range(13)] for s in ["s1", "s2"]}
delta0 = {d: [np.random.binomial(1, 0.7) * np.random.normal(15, 2) for _ in range(13)] for d in ["p1", "p2"]}

env = BlendEnv(v=False, connections = connections, action_sample=action_sample, M=M, P=P, Q=Q, B=B, Z=Z, D=D, tau0 = tau0, delta0 = delta0)
with th.no_grad():
    obs = env.reset()
    obs, obs_dict = obs
    for k in range(env.T):
        
        
        dobs = reconstruct_dict(obs, env.mapping_obs)
        print("\n  OBS     ",dobs["sources"], dobs["blenders"], dobs["demands"], "\n",dobs["sources_avail_next_0"], dobs["demands_avail_next_0"])
        
        obs = th.Tensor(norm_X(obs)).unsqueeze(0)
        action = np.array(model.forward(obs).squeeze(0))
        action = denorm_y(action)
        
        # print(action.shape)
        dact = reconstruct_dict(action, env.mapping_act)
        print("\n\n   ACTION : ")
        print("tau, delta:", dact["tau"], dact["delta"])
        print("source_blend", dact["source_blend"])
        # print("blend_blend", dact["blend_blend"])
        print("blend_demand", dact["blend_demand"])
        
        
        obs, reward, done, term, _ = env.step(action)
        dobs = reconstruct_dict(obs, env.mapping_obs)
        
        print("\n NEW OBS   >>     ",dobs["sources"], dobs["blenders"], dobs["demands"])
        print("   " ,reward)


  OBS      {'s1': 0.0, 's2': 0.0} {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 0.0} {'p1': 0.0, 'p2': 0.0} 
 {'s1': 13.089238, 's2': 12.139851} {'p1': 0.0, 'p2': 17.620289}


   ACTION : 
tau, delta: {'s1': 9.732933921646755, 's2': -4.161153636772497e-06} {'p1': 4.498637139797211, 'p2': 0.0}
source_blend {'s1': {'j1': -7.524426446013659e-06, 'j2': -7.460369857881232e-06, 'j3': 0.6131782528795283, 'j4': 4.858108114672503}, 's2': {'j1': -7.63366364964213e-06, 'j2': -6.437957136284922e-06, 'j3': -4.646376716266332e-06, 'j4': -5.209937414818455e-06}}
blend_demand {'j1': {'p1': 1.1766820162521483, 'p2': 0.0}, 'j2': {'p1': 4.889284331587297, 'p2': 0.0}, 'j3': {'p1': 4.089547565535664, 'p2': 0.0}, 'j4': {'p1': -6.517768014191461e-07, 'p2': -3.545509059676988e-07}}

 NEW OBS   >>      {'s1': 4.2616625, 's2': 1.9766781e-05} {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 4.8581038} {'p1': 0.0, 'p2': 0.0}
    -0.5

  OBS      {'s1': 4.2616625, 's2': 1.9766781e-05} {'j1': 0.0, 'j2': 0.0, 'j3': 0.0, 'j4': 4.8